<a href="https://colab.research.google.com/github/srt3000/Solar-panel-detection/blob/main/solar_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Apr 24 23:30:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 CE903-group_project  'Colab Notebooks'   FireVsNonfire   yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14978, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 14978 (delta 9), reused 22 (delta 7), pack-reused 14949
Receiving objects: 100% (14978/14978), 13.40 MiB | 24.95 MiB/s, done.
Resolving deltas: 100% (10172/10172), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Fire" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-24 23:32:32--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  42.9MB/s    in 4.0s    

2021-04-24 23:32:36 (39.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/resized_frame0.jpg  
  inflating: data/obj/resized_frame0.txt  
  inflating: data/obj/resized_frame10.jpg  
  inflating: data/obj/resized_frame10.txt  
  inflating: data/obj/resized_frame10000.jpg  
  inflating: data/obj/resized_frame10000.txt  
  inflating: data/obj/resized_frame1001.jpg  
  inflating: data/obj/resized_frame1001.txt  
  inflating: data/obj/resized_frame1218.jpg  
  inflating: data/obj/resized_frame1218.txt  
  inflating: data/obj/resized_frame19587.jpg  
  inflating: data/obj/resized_frame19587.txt  
  inflating: data/obj/resized_frame19588.jpg  
  inflating: data/obj/resized_frame19588.txt  
  inflating: data/obj/resized_frame2025.jpg  
  inflating: data/obj/resized_frame2025.txt  
  inflating: data/obj/resized_frame20334.jpg  
  inflating: data/obj/resized_frame20334.txt  
  inflating: data/obj/resized_frame2111.jpg  
 extracting: data/obj/resized_frame2111.txt  
  infla

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.124016 0.438976 0.027559 0.043307

['15', '0.124016', '0.438976', '0.027559', '0.043307']
0 data/obj/resized_frame9999.txt
0 0.124016 0.438976 0.027559 0.043307
15 0.427165 0.226378 0.027559 0.090551

['15', '0.427165', '0.226378', '0.027559', '0.090551']
0 data/obj/resized_frame9999.txt
0 0.427165 0.226378 0.027559 0.090551
15 0.336614 0.287402 0.027559 0.078740

['15', '0.336614', '0.287402', '0.027559', '0.078740']
0 data/obj/resized_frame9999.txt
0 0.336614 0.287402 0.027559 0.078740
15 0.271654 0.226378 0.023622 0.114173

['15', '0.271654', '0.226378', '0.023622', '0.114173']
0 data/obj/resized_frame9999.txt
0 0.271654 0.226378 0.023622 0.114173
15 0.202756 0.608268 0.043307 0.098425

['15', '0.202756', '0.608268', '0.043307', '0.098425']
0 data/obj/resized_frame9999.txt
0 0.202756 0.608268 0.043307 0.098425
15 0.330709 0.872047 0.047244 0.122047

['15', '0.330709', '0.872047', '0.047244', '0.122047']
0 data/obj/resized_frame9999.txt
0 0.330709 0.872047 0.047244 0.122047
15 0

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/resized_frame5211.jpg', 'data/obj/resized_frame5232.jpg', 'data/obj/resized_frame26275.jpg', 'data/obj/resized_frame7246.jpg', 'data/obj/resized_frame6070.jpg', 'data/obj/resized_frame4135.jpg', 'data/obj/resized_frame19588.jpg', 'data/obj/resized_frame10000.jpg', 'data/obj/resized_frame2025.jpg', 'data/obj/resized_frame2592.jpg', 'data/obj/resized_frame27289.jpg', 'data/obj/resized_frame5208.jpg', 'data/obj/resized_frame9998.jpg', 'data/obj/resized_frame22826.jpg', 'data/obj/resized_frame10.jpg', 'data/obj/resized_frame725.jpg', 'data/obj/resized_frame27285.jpg', 'data/obj/resized_frame9999.jpg', 'data/obj/resized_frame0.jpg', 'data/obj/resized_frame27288.jpg', 'data/obj/resized_frame22828.jpg', 'data/obj/resized_frame26272.jpg', 'data/obj/resized_frame2111.jpg', 'data/obj/resized_frame4132.jpg', 'data/obj/resized_frame20334.jpg', 'data/obj/resized_frame5210.jpg', 'data/obj/resized_frame4134.jpg', 'data/obj/resized_frame7248.jpg', 'data/obj/resized_frame24204.jpg', 'data/ob

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.022977, iou_loss = 0.000000, total_loss = 0.022977 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.709358), count: 9, class_loss = 1.116589, iou_loss = 0.548948, total_loss = 1.665537 
 total_bbox = 209388, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.738064), count: 1, class_loss = 0.096941, iou_loss = 0.061618, total_loss = 0.158560 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.693844), count: 14, class_loss = 1.221961, iou_loss = 1.196681, total_loss = 2.418642 
 total_bbox = 209403, rewritten_bbox = 0.000000 % 
v3 (